## Problem-1

### Factorial of Number

In [6]:
import timeit 
x = '''def fact1(N):                         #factorial using for loop
        fact = 1
        for i in range(1,N+1):
            fact = fact * i
        return fact '''       
            
    
y = '''def fact2(N):
        from math import factorial           #factorial using inbuilt function
        return factorial(N)'''

z = '''def fact3(N):                         #factorial by recursion
            if N == 0:
                return 1
            return fact3(N-1) * N'''

print(timeit.timeit(x))
print(timeit.timeit(y))
print(timeit.timeit(z))





0.05114235309883952
0.04385339794680476
0.05726263299584389


In the above question I have compared the time taken to run for the 3 methods of finding the factorial of number N.  
**Method 1**(fact1) : This is method is the most basic method where we iterate through numbers from 1 to N and multiply it with the fact variable. This method has time complexity O(n).  
**Method 2**(fact2) : This is the simplest method where we import the math library and use the in-built **factorial** function. The factorial function use machine code written in C/C++ which is a much quicker language than python. This method has time complexity O(1).  
**Method 3**(fact3) : Here we use a recursive method. We simply call the function with N-1 argument and multiply N with it. This method is memory intensive as fact3(1) till fact(N-1) will be stored until the output is given. This method has time complexity O(n).

From the timeit function we can see that:

1. Method 2 is fastest: It is fastest as it invokes in-built C/C++ machine code which is 45 times faster than python.

2. Method 3 is slowest: It is slowest as it has time complexity O(n). Furthermore it requires for every fact3() to be computed and stored in memory which is a tedious process.

3. Method 1 is in between the above: It has O(n) time complexity and does not C code to carry out the process and is thus slower than method 2. It is not as resource intensive as method 3 and is thus faster than method 3. 

## Problem-2


In [7]:
import numpy as np
#inputs for the 2 matrix solvers
A = np.random.randint(10, size=(10, 10))      #[10,10] Matrix 
b = np.random.randint(10, size=(10, 1))       #[10,1] Matrix

In [8]:
import numpy as np
def gaus(A,b): 
    
    try:
        A = np.array(A, dtype=complex)
        b = np.array(b, dtype=complex)
    
    except (ValueError, NameError):
        return "Please input list of numbers below"
    
    if A.shape[0] != A.shape[1]: #checks if matrix is square
        return "Please enter a square matrix : Number of Rows and Columns of A should be the same"
    
    if A.shape[0] != b.shape[0]: #checks if no. of rows in A and b are the same.
        return "The number of rows in A and b should be the same"
   
    if np.linalg.det(A) == 0:    
        print("No soln or infinite solns")
        
    else:    
        n = len(A)
        index = list(range(n))
        
        for i in range(n):              #arranging matrix in descending order
            temp = i
            for j in range(i+1, n):
                if abs(A[j][i]) > abs(A[temp][i]):
                    temp = j
                    A[[i,temp]] = A[[temp,i]]
                    
                    b[[i,temp]] = b[[temp,i]]
                    
        #making diagonal elements 1 and normalising the other elements accordingly in A and b
        for i in range(n):
            if A[i][i] !=0:              
                ratio = 1.0 / A[i][i]   
                for j in range(n):
                    A[i][j] *= ratio
                b[i] *= ratio
        # making the elements above and below diagonal 1 to zero   
            for k in index[0:i] + index[i+1:]: 
                ratio1 = A[k][i] 
                for j in range(n):
                    A[k][j] = A[k][j] - ratio1 * A[i][j]
                b[k] = b[k] - ratio1 * b[i]
        # A ends up being a identity matrix and b is the solution matrix
        return b
            

%timeit gaus(A,b)


926 µs ± 14.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Matrix solver using Gaussian Elimination
1. Inputs A and b are given to function gaus and stored in float arrays A and b.
2. If input is invalid, eg: character is entered, an error message i given using try-except clause.
3. I use shape function to determine the dimensions of the matrix. This makes sure that input matrix A is square      and A and b have the same number of rows.
4. If determinant of Matrix A is zero, that means that the set of eqns have infinite or no solutions.
5. We go through Matrix A to arrange the rows in descending order wrt values in the first column.
6. We then go row wise to make the pivot(usually diagonal) element 1 by dividing the number by itself(say num n).
7. We also divide the other row elments in A and b by same number n.
8. We then make the non pivot elements as zero in each column by row addition of each row with a multiple of the pivot row. The above is done in A and b accordingly.
9. After the above operations for each row, we get A as an identity matrix(excluding the zero rows at the bottom).
10. The Matrix b is the solution matrix.


In [4]:
import numpy as np
def linal(A,b):

    return np.linalg.solve(A,b)

%timeit linal(A,b)


17.8 µs ± 573 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In the above code, I have used the numpy function to solve the system of equations.
Clearly we can see that the function solves the same set of linear equations almost 200-700 times faster. It runs on C code which is very efficient.
My function gaus() is cumbersome and can be optimized more and runs on python which is quite slow. This is the reason for the stark contrast in time taken for each function.

## Problem 3 : SPICE

In [5]:
import numpy as np   
import math
import cmath
def spice(filename):
      
    file = open(filename,"r")
    file_data = file.readlines()    #read file data into a list of lines
    file.close()
    data_length = len(file_data)
    node_list = []   #list of number of  nodes
    v_count = 0     #no. of independent voltage sources
    node_count = 0
    
    for i in range(data_length):
        line=file_data[i].split()   #Each line is split into words
        
        try:
            if line[0]=='.circuit':     #index of '.circuit' is accessed
                break 
        except IndexError:
            pass
    i_start = i     #index of '.circuit'
    
    
    while 1:
        i += 1
        line=file_data[i].split()   #Each line is split into words
        if line[0]=='.end':    #index of 'end' is accessed
            break
    i_end = i
    
        
    try:             # frequency of source is stored if ac source present. 
        freqline = file_data[i_end+1].split()       
        try:
            freq = float(freqline[2])
            
            
        except ValueError: # to exclude cases when text is below .end
            pass
    except IndexError:     # to exclude cases when there is nothing below .end
        pass
        
    for i in range(i_start+1,i_end):
        line = file_data[i].split()
        node_list.append(line[1])    #We add each node number to node_list 
        node_list.append(line[2])
        if line[0][0] == "V":
            v_count += 1             #Counting voltage sources
            
    uniquenode_list = [*set(node_list)]    #Makes a list with unique node values
    dim = v_count + len(uniquenode_list)   
    
    #matrix A construction
    #We create matrix with dimension as number of nodes(excluding GND) and voltage sources
    matrixA = np.zeros((dim-1,dim-1),dtype = 'complex_') 
    #dim-1 to exclude GND as a node
    #matrixB construction with same number of rows as A
    matrixB = np.zeros((1,dim-1),dtype = 'complex_')
       
    #parsing fle_data to extract all the data and input it in the matrices    
    for i in range(i_start+1,i_end):
        line = file_data[i].split()
        a = line[1] #node1
        b = line[2] #node2
       #adding R values to matrix A 
        if line[0][0] == "R" :
            val = 1/float(line[3])
            if a == "GND" and b!="GND":
                matrixA[int(line[2])-1][int(line[2])-1] += val
                
            elif a != "GND" and b=="GND":
                matrixA[int(line[1])-1][int(line[1])-1] += val 
                
            else:
                matrixA[int(line[1])-1][int(line[1])-1] += val
                matrixA[int(line[2])-1][int(line[2])-1] += val 
                matrixA[int(a)-1][int(b)-1] += val
                matrixA[int(b)-1][int(a)-1] += val
       #adding XL values to matrix A          
        elif line[0][0] == "L":
            val1 = 1/(float(line[3])*2.00j*(math.pi)*freq)
            if a == "GND" and b!="GND":
                matrixA[int(line[2])-1][int(line[2])-1] += val1
                
            elif a != "GND" and b=="GND":
                matrixA[int(line[1])-1][int(line[1])-1] += val1
            else:
                matrixA[int(line[1])-1][int(line[1])-1] += val1
                matrixA[int(line[2])-1][int(line[2])-1] += val1
                matrixA[int(a)-1][int(b)-1] += val1
                matrixA[int(b)-1][int(a)-1] += val1
       #adding XC values to matrix A          
        elif line[0][0] == "C":
            val2 = float(line[3])*2.00j*(math.pi)*freq
            if a == "GND" and b!="GND":
                matrixA[int(line[2])-1][int(line[2])-1] += val2
                
            elif a != "GND" and b=="GND":
                matrixA[int(line[1])-1][int(line[1])-1] += val2  
            
            else:
                matrixA[int(line[1])-1][int(line[1])-1] += val2
                matrixA[int(line[2])-1][int(line[2])-1] += val2
                matrixA[int(a)-1][int(b)-1] += val2
                matrixA[int(b)-1][int(a)-1] += val2
                
        #adding V values to matrix A and b and checking if dc or ac        
        elif line[0][0] == "V" and line[3] == "dc":
            sourcenum = int(line[0][1])
            matrixB[0][len(uniquenode_list) + sourcenum-2] += float(line[4])
            if a == "GND" and b!="GND":
                matrixA[int(b)-1][(len(uniquenode_list))-2 + sourcenum] += -1
                matrixA[(len(uniquenode_list))-2 + sourcenum][int(b)-1] += -1
            elif a != "GND" and b =="GND":
                matrixA[(len(uniquenode_list))-2 + sourcenum][int(a)-1] += 1 
                matrixA[int(a)-1][(len(uniquenode_list))-2 + sourcenum] += 1 
            else:
                matrixA[(len(uniquenode_list))-2 + sourcenum][int(b)-1] += -1
                matrixA[int(b)-1][(len(uniquenode_list))-2 + sourcenum] += -1
                matrixA[(len(uniquenode_list))-2 + sourcenum][int(a)-1] += 1
                matrixA[int(a)-1][(len(uniquenode_list))-2 + sourcenum] += 1
        
        elif line[0][0] == "V" and line[3] == "ac":
            sourcenum = int(line[0][1])
            #converting phasor to rect
            matrixB[0][len(uniquenode_list) + sourcenum-2] += cmath.rect(float(line[4]), float(line[5]))
            
            if a == "GND" and b!="GND":
                matrixA[int(b)-1][(len(uniquenode_list))-2 + sourcenum] += -1
                matrixA[(len(uniquenode_list))-2 + sourcenum][int(b)-1] += -1
            elif a != "GND" and b =="GND":
                matrixA[(len(uniquenode_list))-2 + sourcenum][int(a)-1] += 1 
                matrixA[int(a)-1][(len(uniquenode_list))-2 + sourcenum] += 1 
            else:
                matrixA[(len(uniquenode_list))-2 + sourcenum][int(b)-1] += -1
                matrixA[int(b)-1][(len(uniquenode_list))-2 + sourcenum] += -1
                matrixA[(len(uniquenode_list))-2 + sourcenum][int(a)-1] += 1
                matrixA[int(a)-1][(len(uniquenode_list))-2 + sourcenum] += 1
        #adding I values to matrix A and checking if ac or dc
        elif line[0][0] == "I" and line[3] == "dc":
            if a == "GND" and b!="GND":
                matrixB[0][int(b)-1] -= float(line[4])
                
            elif a != "GND" and b =="GND":
                matrixB[0][int(a)-1] += float(line[4])
                
            else:
                matrixB[0][int(b)-1] += float(line[4])
                matrixB[0][int(a)-1] -= float(line[4])
                
        elif line[0][0] == "I" and line[3] == "ac":
            if a == "GND" and b!="GND":
                matrixB[0][int(b)-1] -= cmath.rect(float(line[4]), float(line[5]))
                
            elif a != "GND" and b =="GND":
                matrixB[0][int(a)-1] += cmath.rect(float(line[4]), float(line[5]))
                
            else:
                matrixB[0][int(b)-1] += cmath.rect(float(line[4]), float(line[5]))
                matrixB[0][int(a)-1] -= cmath.rect(float(line[4]), float(line[5]))
    
    soln = (gaus(matrixA,matrixB[0])) #Solution matrix 
 
    
    for i in range(0,len(uniquenode_list)-1):
        print("Voltage at node " + str(i+1) + " is: " + str(soln[i]))
        
    for i in range(len(uniquenode_list)-1, dim-1):
        print("Current through voltage source " + str(i+1) + " is: " + str(soln[i]))
    
    
spice("ckt2.netlist")    

Voltage at node 1 is: 0j
Voltage at node 2 is: 0j
Voltage at node 3 is: (2+0j)
Voltage at node 4 is: 0j
Voltage at node 5 is: 0j
Voltage at node 6 is: (-65536+0j)
Current through voltage source 7 is: 0j
Current through voltage source 8 is: (-9-0j)


## Circuit Solver using Netlist:

1. We import the relevant libraries: numpy, cmath, math and define spice function(arg: file name)
2. We open the file in read mode, read the data into file_data and close the file.
3. file_data is a list with each element being each line in the file.
4. We then define the start and end points of the circuit using key words ".circuit" and ".end". This makes sure we don't read unneccessary comments above and below the main code.
5. We then find the frequency of any ac source while making sure no errors come for dc circuits using try-except clauses.
6. We then count the number of nodes and independent voltage sources and make the appropriate square zero matrices. Note that * set is used to create a unique list.
7. We now start filling the matrix. The conductances are added to the correct position by reading the nodes, component type(R/L/C: if elif statements are used) and the value of each component. 
8. Note that the nodes should be integer in the netlist.
9. We then add the appropriate voltage sources(dc/ac) to matrix B and add(-1,0 or 1) in matrix A accordingly.
10. Note that cmath was used to convert phasor to rect.
11. The above is done for current sources too.
12. We then plugin our matrix A and b in the gaus matrix solver I built and print out the output values required.



**My reference for MNA** : https://qucs.github.io/tech/node14.html

